<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Analysis/2021_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [ ]:
!pip install -U gensim
!pip install pyLDAvis
!pip install shap
!pip install datasets

import json, os
import pandas as pd
import numpy as np
import nltk
import glob
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
nltk.download('punkt')

# regular expression library
import re
# wordcloud library
from wordcloud import WordCloud
import pprint

import gensim
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

import gensim.corpora as corpora

import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

#Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction                      
import IPython

from gensim.parsing.preprocessing import preprocess_string
from gensim import utils
import gensim.models

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

import shap
import datasets

     |████████████████████████████████| 24.1 MB 2.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a9f15aff89c6b10a88acdfec9505fe17ba9d56e3b0d3c2601aab4fe87c566894
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
     |████████████████████████████████| 564 kB 5.3 MB/s 
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 61.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 13

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/numba/core/types/__init__.py:108: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` t

# Loading and cleaning data

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!dir gdrive/MyDrive/CS/Thesis/Testset_2021/

HetParool_2021.json  LDA_HetParool.png	LDA_Trouw.png
LDA_all.html	     LDA__.html		LDA_Volkskrant.html
LDA_All.html	     LDA_NRC.html	NRC_2021.json
LDA_all.png	     LDA_NRC.png	Telegraaf_2021.json
LDA_All.png	     LDA__.png		Trouw_2021.json
LDA_HetParool.html   LDA_Trouw.html	Volkskrant_2021.json


In [ ]:
path_NRC = 'gdrive/MyDrive/CS/Thesis/Testset_2021/NRC_2021.json'
path_Trouw = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Trouw_2021.json'
path_HetParool = 'gdrive/MyDrive/CS/Thesis/Testset_2021/HetParool_2021.json'
path_Telegraaf = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Telegraaf_2021.json'
path_Volkskrant = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Volkskrant_2021.json'

In [ ]:
df_NRC = pd.read_json(path_NRC)
df_Trouw = pd.read_json(path_Trouw)
df_HetParool = pd.read_json(path_HetParool)
df_Telegraaf = pd.read_json(path_Telegraaf)
df_Volkskrant = pd.read_json(path_Volkskrant)

In [ ]:
df_NRC["Source"] = "NRC"
df_Trouw["Source"] = "Trouw"
df_HetParool["Source"] = "HetParool"
df_Telegraaf["Source"] = "Telegraaf"
df_Volkskrant["Source"] = "Volkskrant"

In [ ]:
print(df_NRC.shape)
print(df_Trouw.shape)
print(df_HetParool.shape)
print(df_Volkskrant.shape)

(38334, 7)
(21077, 7)
(18551, 7)
(25773, 7)


## Concat data

In [ ]:

dfs = [df_NRC, df_Trouw, df_HetParool, df_Volkskrant]
df = pd.concat(dfs, ignore_index=True)

df.head()

,url,timestamp,title,publisherID,cleantext,category,Source
0,https://www.nrc.nl/nieuws/2021/01/31/een-herha...,2021-01-31 20:18:43,Een herhaling van vorig seizoen is zo goed als...,Fabian van der Poll,De eerste maand van 2021 is uitstekend verlope...,None,NRC
1,https://www.nrc.nl/nieuws/2021/01/31/avondklok...,2021-01-31 20:02:10,‘Avondklok waarschijnlijk niet verlengd als be...,Steven Musch,De avondklok wordt waarschijnlijk niet verleng...,None,NRC
2,https://www.nrc.nl/nieuws/2021/01/31/school-op...,2021-01-31 19:58:21,"School open? Dan testen, ventilatie en mondkapjes",Karel Berkhout,Door de heropening van de scholen loopt het aa...,None,NRC
3,https://www.nrc.nl/nieuws/2021/01/31/scholen-o...,2021-01-31 19:45:18,"De basisscholen gaan open, met een paar nieuwe...",Claudia Kammer,"Het werd alom verwacht, en zondag aan het ei...",None,NRC
4,https://www.nrc.nl/nieuws/2021/01/31/arrestati...,2021-01-31 19:43:49,Arrestatie Russische betoger,None,,None,NRC


## Remove missing text

In [ ]:
df = df[df['cleantext'].notna()]
print(df.shape)

(103723, 7)


# LDA

In [ ]:
def lda_analysis(frame, path_lda, topics=10):
  # Remove the columns
  articles = frame.drop(columns=['url', 'publisherID', 'category',], axis=1)

  articles = articles[articles['cleantext'].notna()]

  # Remove punctuation
  articles['cleantext_processed'] = articles['cleantext'].map(lambda x: re.sub('[,\\.!?]', '', x))
  # Save D66
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('D66', 'dzeszes', x))
  # Save BIJ1
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('BIJ1', 'bijeeen', x))
  # Save 50PLUS
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('50PLUS', 'vijfplus', x))
  # Save JA21
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('JA21', 'jatweeeen', x))
  # Save DenHaan
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('Den Haan', 'denhaan', x))
  # Convert the titles to lowercase
  articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: x.lower())

  p_and_p =  ['vvd', 'dzeszes', 'pvv', 'cda', 'pvda', 'sp', 'gl', 'fvd', 'pvdd',
             'cu', 'sgp', 'volt', 'jatweeeen', 'bbb', 'bijeeen', 'vijfplus', 'rutte', 'kaag',
             'wilders', 'hoekstra', 'ploumen', 'marijnissen', 'klaver', 'baudet',
             'ouwehand', 'segers', 'staaij', 'dassen', 'eerdmans', 'plas',
             'simons', 'denhaan']

  
  pat = '|'.join(r"\b{}\b".format(x) for x in p_and_p)
  articles = articles[articles['cleantext_processed'].str.contains(pat)]
  print(articles.shape)

  stop_words.extend(['nbsp', 'rsquo', 'lsquo','euml', 'rdquo', 'quot', 'ldquo',
                     'we', 'wel', 'jaar', 'zegt', 'gaat','moeten', 'volgens',
                     'heel', 'maken', 'weer', 'gaan', 'twee', 'mensen', 'waar', 'alleen',
                     'komt', 'zoals', 'alle', 'wij', 'zei','komen','tussen', 'eerste','zien'
                     'eigen', 'leven','grote', 'nieuwe','mee','goed', 'aantal','laten'
                     ,'tweede', 'af', 'eerder', 'eigen', 'elkaar','afgelopen', 'week',
                     'zeggen', 'echt', 'dag', 'zien', 'steeds', 'laten', 'willen', 'waarin'])

  def sent_to_words(sentences):
      for sentence in sentences:
          # deacc=True removes punctuations
          yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) 
              if word not in stop_words] for doc in texts]

  data = articles.cleantext_processed.values.tolist()
  data_words = list(sent_to_words(data))# remove stop words
  data_words = remove_stopwords(data_words)

  # Create Dictionary
  id2word = corpora.Dictionary(data_words)

  # Create Corpus
  texts = data_words

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  #print(corpus[:1][0][:30])

  # number of topics
  num_topics = topics
  # Build LDA model
  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=num_topics)
  pprint.pprint(lda_model.show_topics(num_words=20, formatted=False))
  doc_lda = lda_model[corpus]

  # def terms_topics_frame(terms):

  #   df = pd.DataFrame(data=None)

  #   for t in terms:
  #     df_t_topics = pd.DataFrame(lda_model.get_term_topics(t, minimum_probability=0.000000001))
  #     df_t_topics = df_t_topics.drop(columns=0)
  #     df_t_topics = df_t_topics.rename(columns={1:t})
  #     df = pd.concat([df, df_t_topics], axis=1)
    
  #   return df

  # df_tt = terms_topics_frame(['vvd','dzeszes','pvv','cda', 'pvda',  'sp','groenlinks','fvd','pvdd','cu',
  #                               'sgp','volt','jatweeeen','bbb', 'bijeeen', 'vijfplus']
  # )
  # df_tt['topic'] = df_tt.index

  # sns.set_theme(style="whitegrid")

  # # Plot the topic dist per term
  # fig = df_tt.set_index('topic').plot(kind='bar', figsize=(20,20), stacked=True)
  # fig.figure.savefig(path_plot)

  # Visualize the topics
  pyLDAvis.enable_notebook()
  LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
  LDAvis = gensimvis.prepare(lda_model, corpus, id2word, sort_topics=False)
  pyLDAvis.save_html(LDAvis, path_lda) 


In [ ]:
lda_analysis(df,'gdrive/MyDrive/CS/Thesis/Testset_2021/LDA_All.html',topics=10)

(15498, 5)
[(0,
  [('euro', 0.005008146),
   ('procent', 0.0038205648),
   ('gemeenten', 0.0035917547),
   ('miljoen', 0.0025716557),
   ('museum', 0.0021115723),
   ('per', 0.0019279716),
   ('kabinet', 0.0019161823),
   ('nederland', 0.001855373),
   ('cda', 0.0018453025),
   ('kamer', 0.0017862418),
   ('partij', 0.001772817),
   ('nederlandse', 0.0017379906),
   ('staat', 0.0017271027),
   ('stemmen', 0.0016917441),
   ('gemeente', 0.0016549564),
   ('den', 0.001505172),
   ('geld', 0.0014911494),
   ('the', 0.0014020349),
   ('weten', 0.0013934639),
   ('vanwege', 0.0013679349)]),
 (1,
  [('euro', 0.005312666),
   ('miljoen', 0.0033961108),
   ('kabinet', 0.0032385692),
   ('onderwijs', 0.0029735782),
   ('nederland', 0.0027904483),
   ('geld', 0.002501123),
   ('krijgen', 0.00223317),
   ('procent', 0.0020428796),
   ('per', 0.0019657474),
   ('mogelijk', 0.001860617),
   ('kinderen', 0.0018564861),
   ('minister', 0.0018306205),
   ('werk', 0.0018268517),
   ('uur', 0.0017892737

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
lda_analysis(df_NRC,'gdrive/MyDrive/CS/Thesis/Testset_2021/LDA_NRC.html',topics=10)

(6423, 5)
[(0,
  [('nederland', 0.0035682733),
   ('cda', 0.0025780397),
   ('kabinet', 0.0025671166),
   ('procent', 0.0023296056),
   ('rivm', 0.0022552616),
   ('kamer', 0.0022334948),
   ('nederlandse', 0.0019532095),
   ('vvd', 0.0019516306),
   ('rutte', 0.0019166128),
   ('partijen', 0.0018022698)]),
 (1,
  [('procent', 0.003904058),
   ('nederland', 0.0034715945),
   ('kabinet', 0.0034515315),
   ('minder', 0.0033605087),
   ('nederlandse', 0.002869975),
   ('europese', 0.0024860909),
   ('minister', 0.002318721),
   ('staat', 0.002133003),
   ('vvd', 0.0021308865),
   ('mogelijk', 0.0020686623)]),
 (2,
  [('nederland', 0.0040096613),
   ('minister', 0.003836529),
   ('nederlandse', 0.0024946625),
   ('rutte', 0.0024907745),
   ('kabinet', 0.0024695215),
   ('bergkamp', 0.0021612532),
   ('procent', 0.0019058068),
   ('kamer', 0.0018866373),
   ('jonge', 0.0018844539),
   ('staat', 0.001819377)]),
 (3,
  [('nederland', 0.0041498044),
   ('rutte', 0.0035841395),
   ('kabinet', 0

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
lda_analysis(df_Trouw,'gdrive/MyDrive/CS/Thesis/Testset_2021/LDA_Trouw.html',topics=10)

(2874, 5)
[(0,
  [('rutte', 0.0041611884),
   ('nederland', 0.0040403884),
   ('kabinet', 0.0033240942),
   ('procent', 0.0024453965),
   ('kamer', 0.0023337828),
   ('vooral', 0.0020462896),
   ('tijd', 0.001989405),
   ('europese', 0.0019640592),
   ('staat', 0.001961271),
   ('minister', 0.001788929)]),
 (1,
  [('kabinet', 0.005093189),
   ('rutte', 0.004181263),
   ('nederland', 0.0036956056),
   ('vvd', 0.002971661),
   ('minister', 0.0029162522),
   ('staat', 0.0023370201),
   ('cda', 0.002266663),
   ('kamer', 0.002051446),
   ('nederlandse', 0.0020212538),
   ('tijd', 0.0018086395)]),
 (2,
  [('kabinet', 0.0042751785),
   ('dzeszes', 0.004261955),
   ('rutte', 0.0041182283),
   ('nederland', 0.0038251367),
   ('cda', 0.0034531099),
   ('kamer', 0.0032858667),
   ('vvd', 0.0030815443),
   ('minister', 0.002939789),
   ('staat', 0.0020949515),
   ('procent', 0.001967895)]),
 (3,
  [('rutte', 0.0053786947),
   ('kabinet', 0.0052471473),
   ('nederland', 0.0037805147),
   ('staat',

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
lda_analysis(df_HetParool,'gdrive/MyDrive/CS/Thesis/Testset_2021/LDA_HetParool.html',topics=10)

(2452, 5)
[(0,
  [('rutte', 0.005178491),
   ('amsterdam', 0.0033900444),
   ('kabinet', 0.0033574884),
   ('vvd', 0.0032680521),
   ('nederland', 0.0027269346),
   ('staat', 0.0022320617),
   ('dzeszes', 0.0020053738),
   ('procent', 0.0019337855),
   ('cda', 0.0018497654),
   ('vooral', 0.0018235937)]),
 (1,
  [('kabinet', 0.004078071),
   ('nederland', 0.002923011),
   ('rutte', 0.0025618616),
   ('cda', 0.0023626054),
   ('amsterdam', 0.0020637515),
   ('minister', 0.002000617),
   ('partij', 0.0019241751),
   ('onze', 0.0018462337),
   ('kamer', 0.0017946082),
   ('iedereen', 0.0017815474)]),
 (2,
  [('rutte', 0.0036947797),
   ('amsterdam', 0.0030278247),
   ('kabinet', 0.0029372491),
   ('partij', 0.0027156917),
   ('nederland', 0.0024631),
   ('pvda', 0.0022879157),
   ('dzeszes', 0.0021994743),
   ('partijen', 0.0021939767),
   ('cda', 0.0020006762),
   ('euro', 0.00197722)]),
 (3,
  [('kabinet', 0.003796491),
   ('nederland', 0.0032306283),
   ('kamer', 0.0030112548),
   ('pa

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
